# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/danielgarciav010@gmail.com/DS_jobs.csv")

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

spark = (
    SparkSession
    .builder
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.15.0")
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

In [0]:
dataframe = spark.read.option("header", "true") \
               .option("delimiter", ",") \
               .option("multiline", "true") \
               .option("quote", "\"") \
               .option("escape", "\"") \
               .csv("dbfs:/FileStore/shared_uploads/danielgarciav010@gmail.com/DS_jobs.csv")

dataframe.printSchema()

2. Eliminar duplicados

In [0]:
dataframe = dataframe.dropDuplicates()

3. Decidir que hacer con los datos faltantes 

In [0]:
dataframe = dataframe.drop("index")

dataframe.display()

4. Decidir que hacer con los valores nulos

In [0]:
dataframe = dataframe.na.drop(subset=["Job Title"])

5. ¿Cuántos registros tiene el csv?

In [0]:
dataframe.count()

6. Mostrar los valores únicos de `Job title` 

In [0]:
dataframe.select("Job title").distinct().display()

9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
# Elimina todo lo que esté después de ( del campo Salary Estimate
dataframe = dataframe.withColumn("Salary Estimate", split(dataframe["Salary Estimate"], "\(")[0])
dataframe.display()

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
# Elimina la k de Salary Estimate
dataframe = dataframe.withColumn("Salary Estimate", regexp_replace(lower(dataframe["Salary Estimate"]), "k", ""))

# Elimina el $ de Salary Estimate
dataframe = dataframe.withColumn("Salary Estimate", regexp_replace(lower(dataframe["Salary Estimate"]), "\$", ""))

# Saca la media entre las dos cifras y las multiplica por 1000 por la sustitución de la k
dataframe = dataframe.withColumn("Salary Estimate",  concat( (split(dataframe["Salary Estimate"], "-")[0] * 1000), lit(" - "), (split(dataframe["Salary Estimate"], "-")[1] * 1000)))

dataframe.display()

8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
dataframe.select("Salary Estimate").distinct().display()

10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
# Muestra los salarios de mayor a menor y la cantidad de empleos que ofrecen este sueldo ( Me parece una información más útil que lo que devuelve la consulta anterior )
dataframe.groupBy("Salary Estimate").count().sort("Salary Estimate", ascending=False).display()

11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
dataframe = dataframe.withColumn("Job Description", regexp_replace(dataframe["Job Description"], "\\n", ""))

dataframe.display()

12. De la columna `Rating` muestre los valores unicos.

In [0]:
dataframe.select("Rating").distinct().display()

13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
dataframe = dataframe.withColumn("Rating", regexp_replace(dataframe.Rating, "-1", "0.0"))

dataframe.select("Rating").distinct().display()

14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
dataframe.select("Company Name").distinct().sort("Company Name", ascending=True).display()

15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
# Se carga el salto de línea y el número (e.j 4.6) de los nombres de compañía que lo tuviesen
dataframe = dataframe.withColumn("Company Name", split(dataframe["Company Name"], "\\n")[0])

16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
dataframe = dataframe.withColumn("City", split(dataframe.Location, ", ")[0])
dataframe = dataframe.withColumn("State", split(dataframe.Location, ", ")[1])
dataframe = dataframe.drop("Location")
dataframe.display()

17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
dataframe = dataframe.withColumn("Headquarter State", split(dataframe.Headquarters, ", ")[1])
dataframe = dataframe.withColumn("Headquarters", split(dataframe.Headquarters, ", ")[0])
dataframe.display()

18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
dataframe.select("Headquarter State").distinct().display()

19. Mostrar valores unicos del campo `Size`.

In [0]:
dataframe.select("Size").distinct().display()

20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
dataframe = dataframe.withColumn("Size", regexp_replace(dataframe.Size, " employees", ""))

21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
dataframe = dataframe.withColumn("Size", regexp_replace(dataframe.Size, "to", "-"))
dataframe = dataframe.withColumn("Size", regexp_replace(dataframe.Size, "-1", "Unknown"))

22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
dataframe.select("Type of ownership").distinct().display()

23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
dataframe = dataframe.withColumn("Type of ownership", regexp_replace(dataframe["Type of ownership"], "-1", "Unknown"))

dataframe.select("Type of ownership").distinct().display()

24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
dataframe = dataframe.withColumn("Type of ownership", regexp_replace(dataframe["Type of ownership"], "Company - Public", "Public Company"))
dataframe = dataframe.withColumn("Type of ownership", regexp_replace(dataframe["Type of ownership"], "Company - Private", "Private Company"))
dataframe = dataframe.withColumn("Type of ownership", regexp_replace(dataframe["Type of ownership"], "Private Practice / Firm", "Private Company"))
dataframe = dataframe.withColumn("Type of ownership", regexp_replace(dataframe["Type of ownership"], "Subsidiary or Business Segment", "Business"))
dataframe = dataframe.withColumn("Type of ownership", regexp_replace(dataframe["Type of ownership"], "College / University", "Education"))

dataframe.select("Type of ownership").distinct().display()

25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
print(dataframe.select("Industry").dtypes)
dataframe.select("Industry").distinct().display()

26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
dataframe = dataframe.withColumn("Industry", regexp_replace(dataframe.Industry, "-1", "Not Available"))
dataframe = dataframe.withColumn("Industry", regexp_replace(dataframe.Industry, "\&", "and"))

dataframe.select("Industry").distinct().sort("Industry", ascending=True).display()

27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
print(dataframe.select("Sector").dtypes)
dataframe.select("Sector").distinct().display()

28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
dataframe = dataframe.withColumn("Sector", regexp_replace(dataframe.Sector, "-1", "Not Available"))
dataframe = dataframe.withColumn("Sector", regexp_replace(dataframe.Sector, "\&", "and"))

dataframe.select("Sector").distinct().sort("Sector", ascending=True).display()

29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
print(dataframe.select("Revenue").dtypes)
dataframe.select("Revenue").sort("Revenue", ascending=True).distinct().display()

30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
dataframe = dataframe.withColumn("Revenue", regexp_replace(dataframe.Revenue, "-1", "N/A"))
dataframe = dataframe.withColumn("Revenue", regexp_replace(dataframe.Revenue, "Unknown / Non-Applicable", "N/A"))
dataframe = dataframe.withColumn("Revenue", regexp_replace(dataframe.Revenue, "Less than $1 million (USD)", "Less than 1"))
dataframe = dataframe.withColumn("Revenue", regexp_replace(dataframe.Revenue, "\$", ""))
dataframe = dataframe.withColumn("Revenue", regexp_replace(dataframe.Revenue, "\(USD\)", ""))


dataframe.select("Revenue").distinct().sort("Revenue", ascending=True).display()

31. Borrar el campo `Competitors`.

In [0]:
dataframe = dataframe.drop("Competitors")

32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
dataframe = dataframe.withColumn("min_salary", split(dataframe["Salary Estimate"], " - ")[0])
dataframe = dataframe.withColumn("max_salary", split(dataframe["Salary Estimate"], " - ")[1])
dataframe = dataframe.withColumn("avg_salary", (dataframe.min_salary+dataframe.max_salary)/2)

dataframe.display()

33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
print(dataframe.select("Founded").dtypes)
dataframe.select("Founded").distinct().display()

34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
dataframe = dataframe.withColumn("Founded", regexp_replace(dataframe.Founded, "-1", "2024"))

dataframe.select("Founded").distinct().display()

35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

In [0]:
dataframe = dataframe.withColumn("company_age", datetime.now().year - dataframe.Founded)

dataframe.display()

36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

In [0]:
dataframe = dataframe.withColumn("Job Type", when(
  (
    lower(dataframe["Job Title"]).contains("sr") |
    lower(dataframe["Job Title"]).contains("senior") |
    lower(dataframe["Job Title"]).contains("lead") |
    lower(dataframe["Job Title"]).contains("principal") |
    lower(dataframe["Job Title"]).contains("experienced") |
    lower(dataframe["Job Title"]).contains("expert") 
  ),
  lit("Senior")
).otherwise(
        when(
            (
                lower(dataframe["Job Title"]).contains("junior") |
                lower(dataframe["Job Title"]).contains("jr")
            ),
            lit("Junior")
        ).otherwise(
          lit("NA")
        )
))

dataframe.display()


37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
dataframe.select("Job Type").distinct().display()

38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

In [0]:
dataframe = dataframe.withColumn("Python", (lower(dataframe["Job Description"]).contains(" python ") | lower(dataframe["Job Description"]).contains(" py ")))
dataframe = dataframe.withColumn("Spark", lower(dataframe["Job Description"]).contains(" spark "))
dataframe = dataframe.withColumn("Big Data", lower(dataframe["Job Description"]).contains(" big data "))
dataframe = dataframe.withColumn("Scala", lower(dataframe["Job Description"]).contains(" scala "))
dataframe = dataframe.withColumn("R", lower(dataframe["Job Description"]).contains(" r "))
dataframe = dataframe.withColumn("Java", lower(dataframe["Job Description"]).contains(" java "))
dataframe = dataframe.withColumn("Excel", lower(dataframe["Job Description"]).contains(" excel "))
dataframe = dataframe.withColumn("Machine Learning", (lower(dataframe["Job Description"]).contains(" machine learning ") | lower(dataframe["Job Description"]).contains(" ml ")))
dataframe = dataframe.withColumn("Azure", lower(dataframe["Job Description"]).contains(" azure "))
dataframe = dataframe.withColumn("AWS", lower(dataframe["Job Description"]).contains(" aws "))
dataframe = dataframe.withColumn("Databricks", lower(dataframe["Job Description"]).contains(" databricks "))
dataframe = dataframe.withColumn("Hadoop", lower(dataframe["Job Description"]).contains(" hadoop "))


dataframe.display()

Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


39. Exportar dataset final a csv

In [0]:
dataframe.write.csv("dbfs:/FileStore/shared_uploads/danielgarciav010@gmail.com/caso_4/jobs_clean.csv", mode="overwrite", header=True)

In [0]:
spark.read.csv("dbfs:/FileStore/shared_uploads/danielgarciav010@gmail.com/caso_4/jobs_clean.csv", header=True).display()

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 

In [0]:
dataframe.printSchema()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Cambia a double las columnas de salario
dataframe = dataframe.withColumn("min_salary", col("min_salary").cast("double"))
dataframe = dataframe.withColumn("max_salary", col("max_salary").cast("double"))
dataframe = dataframe.withColumn("avg_salary", col("avg_salary").cast("double"))

# Pinta el gráfico con el número de ofertas para cada nivel de experiencia
job_type_count = dataframe.groupBy("Job Type").count().toPandas()
sns.barplot(x="Job Type", y="count", data=job_type_count)
plt.title("Job Type Distribution")
# plt.show()

## En este gráfico se ve que no hay muchas ofertas de trabajo específicamente buscando  juniors. Pero eso no significa que casi no haya oportunidades para ellos, ya que muchos puestos clasificados como 'NA' podrían ser para juniors. Estos puestos no dicen que no busquen juniors, solo que están buscando a alguien en general, sin especificar el nivel de experiencia.

#### En el csv solo hay una oferta para junior, esto va a afectar a todas las gráficas, sería necesario encontrar más ofertas para junior

In [0]:
# Pinta el gráfico con la media de salarios entre junior y senior
avg_salary_by_type = dataframe.groupBy("Job Type").avg("avg_salary").toPandas()
sns.barplot(x="Job Type", y="avg(avg_salary)", data=avg_salary_by_type)
plt.title("Average Salary by Job Type")
# plt.show()

## Lo que podemos extraer de este gráfico es que por lo general, los senior tienen un salario de casi 1,5x respecto a lo que cobra un junior

In [0]:
tech_columns = [
    "Python", "Spark", "Big Data", "Scala", "R", "Java", 
    "Excel", "Machine Learning", "Azure", "AWS", "Databricks", "Hadoop"
]

# Calcula las ofertas por tecnología
tech_counts = {}
for tech in tech_columns:
    tech_counts[tech] = dataframe.filter(col(tech) == True).count()

# Pinta el gráfico con las ofertas que piden cada tecnología
tech_df = pd.DataFrame(list(tech_counts.items()), columns=["Technology", "Count"])
sns.barplot(x="Technology", y="Count", data=tech_df)
plt.title("Tech Requirements Count")
plt.xticks(rotation=45)
plt.show()


## En este gráfico podemos ver que la tecnología más explícitamente demandada entre las ofertas es Machine Learning seguida de Python

In [0]:
avg_salary_tech = []

# Calcula el salario promedio para cada tipo de trabajo y habilidad técnica, luego pivota los datos
for job_type in dataframe.select("Job Type").distinct().rdd.flatMap(lambda x: x).collect():
    for tech in tech_columns:
        avg_salary = dataframe.filter((dataframe["Job Type"] == job_type) & (dataframe[tech] == True))\
                               .agg({"avg_salary": "avg"}).collect()[0][0]
        avg_salary_tech.append((job_type, tech, avg_salary))

heatmap_df = pd.DataFrame(avg_salary_tech, columns=["Job Type", "Tech Skill", "Avg Salary"])

heatmap_df_pivot = heatmap_df.pivot("Tech Skill", "Job Type", "Avg Salary")

# Pinta un mapa de calor que relaciona la experiencia, con la tencnología que se pide y el salario medio
plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_df_pivot, annot=True, cmap="YlGnBu", fmt=".1f", cbar_kws={'label': 'Average Salary'}, linewidths=0.5)
plt.title("Heatmap of Average Salary by Job Type and Tech Skills", fontsize=16)
plt.xlabel("Job Type", fontsize=12)
plt.ylabel("Tech Skills", fontsize=12)
plt.xticks(rotation=45)
plt.show()

## En este gráfico podemos observar que tecnología se paga más por el nivel de experiencia. Podemos ver que Un senior de java es lo mejor pagado entre el listado de empleos mientras que un senior de spark es lo que menos cobra

In [0]:
salary_tech_df = []


for job_type in dataframe.select("Job Type").distinct().rdd.flatMap(lambda x: x).collect():

    job_data = dataframe.filter(dataframe["Job Type"] == job_type).toPandas()
    
    # Para cada tecnología, crea una columna indicando si es requerida o no
    for tech in tech_columns:
        job_data["Tech Skill"] = job_data[tech].apply(lambda x: "Required" if x else "Not Required")
        job_data["Tech Skill"] = job_data["Tech Skill"].astype('category')  # Ensure it's a category for hue
        salary_tech_df.append(job_data[["Job Type", "avg_salary", "Tech Skill"]])


salary_tech_df = pd.concat(salary_tech_df)

# Pinta el gráfico que relaciona el tipo de experiencia con el salario y si requiere conocimientos en alguna de las tecnologías del dataframe
plt.figure(figsize=(14, 8))
sns.violinplot(x="Job Type", y="avg_salary", hue="Tech Skill", data=salary_tech_df, split=True, inner="quart", palette="muted")

plt.title("Salary Distribution by Job Type and Tech Skill Requirement", fontsize=16)
plt.xlabel("Job Type", fontsize=12)
plt.ylabel("Average Salary ($)", fontsize=12)
plt.xticks(rotation=45)
plt.show()

# En este gráfico podemos observar:

## Los trabajos senior tienen salarios promedio más altos que los trabajos junior o NA, independientemente de si requieren habilidades técnicas.

## Las habilidades técnicas tienden a incrementar los salarios en los trabajos senior, pero no parecen influir significativamente en la categoría NA.

## Sabiendo esto, si somos una empresa quizá nos pueda interesar no ofertar buscando senior ni junior, de esta forma podemos reducir en costes de contratación

In [0]:
dataframe.display()

## En este gráfico podemos observar los salarios medios en cada estado de USA

In [0]:
display(dataframe)

## Este gráfico muestra los salarios de cada puesto en las distintas empresas

## Podemos observar que los puestos que más pagan son como Data Scientist, en industrias como:
- Enterprise Software and Network Solutions
- Publishing
- Health, Beauty, and Fitness
- Wholesale
- State and Regional Agencies
- Hotels, Models and Resorts